In [ ]:
pip install apache-tvm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 44.2 MB 1.8 MB/s 


In [ ]:
import numpy as np
from google.colab import files
from google.colab import drive

drive.mount('/content/drive')
path = '/content/drive/MyDrive/Colab Notebooks/relay/' # the absolute path to your file
%cd /content/drive/MyDrive/Colab Notebooks/relay/
!ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/relay
184.jpg  b.csv	 d.csv	      test1.jpg       Untitled0.ipynb
b1.csv	 d1.csv  relay.ipynb  test_image.jpg


In [ ]:
from matplotlib.image import imread
import pandas as pd
import tvm
from tvm import relay
from tvm.contrib import graph_executor
import cv2

In [ ]:
d = tvm.nd.array(pd.read_csv('d.csv',header=None).to_numpy().astype("float32"))
b = tvm.nd.array(pd.read_csv('b.csv',header=None).to_numpy().astype("float32"))
d1 = tvm.nd.array(pd.read_csv('d1.csv',header=None).to_numpy().astype("float32"))
b1 = tvm.nd.array(pd.read_csv('b1.csv',header=None).to_numpy().astype("float32"))
image = cv2.imread('184.jpg',cv2.IMREAD_GRAYSCALE).flatten().reshape(1,784)

In [ ]:
import numpy as np
import tvm
from tvm import relay
from tvm.contrib import graph_executor

def get_network_architecture(image: np.ndarray, d: tvm.nd.array, b: tvm.nd.array, d1: tvm.nd.array, b1: tvm.nd.array):
    dtype="float32"
    ###
    # Your code starts here...
    # Step 1: Create relay input variable via relay.var("input", shape=..., dtype=...)
    input = tvm.relay.var("input", shape=[1, 28*28], dtype=dtype)

    # Step 2: Create relay variables for weights via weight1 = relay.var("d", shape=..., dtype=...),
    # weights names list: d - first dense layer, b - first bias, d1 - second dense layer, b1 - second bias
    weight1 = relay.var("weight1", shape=d.shape, dtype=dtype)
    weight2 = relay.var("weight2", shape=b.shape, dtype=dtype)
    weight3 = relay.var("weight3", shape=d1.shape, dtype=dtype)
    weight4 = relay.var("weight4", shape=b1.shape, dtype=dtype)

    # Step 3: Write your architecture. In our example case:
    # %0 transpose (dense weights)
    # %1 dense layer (input, %0)
    # %2 bias add (%1, bias weights)
    # %3 relu (%2)
    # %4 transpose (dense2 weights)
    # %5 dense2 layer (%3, %4)
    # %6 bias add (%5, bias 2 weights) -> softmax (%6)

    step_0 = relay.transpose(weight1)
    step_1 = relay.nn.dense(input, step_0)
    step_2 = relay.nn.bias_add(step_1, relay.transpose(weight2))
    step_3 = relay.nn.relu(step_2)
    step_4 = relay.transpose(weight3)
    step_5 = relay.nn.dense(step_3, step_4)
    step_6 = relay.nn.bias_add(step_5, relay.transpose(weight4))
    sm = relay.nn.softmax(step_6)

    # Step 4: Dont forget about params! Enter all weights names there.
    # Example: (params = {weight1: "d", weight2: "...", ..})
    params = {"weight1": d , "weight2": b, "weight3": d1, "weight4": b1}
    #params = {d: "d", b: "b", d1: "d1", b1: "b1"}
    # Step 5: Get mod via relay.Function([input, weight1, weight2, weight3, weight4], last_layer_name)
    mod = relay.Function([input, weight1, weight2, weight3, weight4], sm)

    return mod, params

In [ ]:
import numpy as np
import tvm
from tvm import relay
from tvm.contrib import graph_executor

def get_network_architecture(image: np.ndarray, d: tvm.nd.array, b: tvm.nd.array, d1: tvm.nd.array, b1: tvm.nd.array):
    dtype="float32"
    ###
    # Your code starts here...
    # Step 1: Create relay input variable via relay.var("input", shape=..., dtype=...)
    input = tvm.relay.var("input", relay.TensorType(image.shape,dtype))

    # Step 2: Create relay variables for weights via weight1 = relay.var("d", shape=..., dtype=...),
    # weights names list: d - first dense layer, b - first bias, d1 - second dense layer, b1 - second bias
    dense = relay.var("d", shape=d.shape, dtype=dtype)
    bias = relay.var("b", shape=b.shape, dtype=dtype)
    dense1 = relay.var("d1", shape=d1.shape, dtype=dtype)
    bias1 = relay.var("b1", shape=b1.shape, dtype=dtype)

    # Step 3: Write your architecture. In our example case:
    # %0 transpose (dense weights)
    # %1 dense layer (input, %0)
    # %2 bias add (%1, bias weights)
    # %3 relu (%2)
    # %4 transpose (dense2 weights)
    # %5 dense2 layer (%3, %4)
    # %6 bias add (%5, bias 2 weights) -> softmax (%6)
    dense_T=relay.transpose(dense)
    dense1_T=relay.transpose(dense1)

    net = relay.nn.dense(input, dense_T)
    net = relay.add(net, relay.transpose(bias))
    net = relay.nn.relu(net)
    net = relay.nn.dense(net, dense1_T)
    net = relay.add(net, relay.transpose(bias1))
    net = relay.nn.softmax(net)

    # Step 4: Dont forget about params! Enter all weights names there.
    # Example: (params = {weight1: "d", weight2: "...", ..})
    params = {"d": d , "b": b, "d1": d1, "b1": b1}
    #params = {d: "d", b: "b", d1: "d1", b1: "b1"}
    # Step 5: Get mod via relay.Function([input, weight1, weight2, weight3, weight4], last_layer_name)
    mod = relay.Function([input, dense, bias, dense1, bias1], net)

    return mod, params

In [ ]:
mod, params = get_network_architecture(image=image,d=d,b=b,d1=d1,b1=b1)

In [ ]:
def test_model(img_tvm, mod, params, target="llvm"):
  with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mod, target=target, params=params)

  #dev = tvm.cpu(0)
  dev = tvm.device(target, 0)

  gmod = graph_executor.GraphModule(lib["default"](dev))
  gmod.set_input("input", img_tvm)
  gmod.run()
  tvm_output = gmod.get_output(0)
  output = tvm_output.asnumpy()
  pred = np.argmax(output)
  return pred, output

In [ ]:
pred,output=test_model(image,mod,params)

In [ ]:
print(pred,output)

8 [[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]]
